# PR0402: MapReduce (II)

In [1]:
%cd pro402

/media/notebooks/pro402


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


# Ejercicio 1: temperatura máxima por ciudad

In [2]:
%%writefile mapper1.py
#!/usr/bin/env python3

import sys


first = True

for line in  sys.stdin:
    
    if first == True:
        first = False
        continue

    line = line.strip().split(',')
    print(f"{line[3]}\t{line[7]}")    



Overwriting mapper1.py


In [3]:
%%writefile reducer1.py
#!/usr/bin/env python3
 
import sys

max_temp = -100.0
ciudad = None
ciudad_actual = None
lineafinal = ""

for line in sys.stdin:
    line = line.split('\t',1)

    ciudad = line[0]
    temp = float(line[1])

    if ciudad_actual is None:
        ciudad_actual = ciudad
        
    if temp > max_temp:
        max_temp = temp
        lineafinal = f"{ciudad_actual}  --->  {temp}"

    if ciudad_actual != ciudad:
        print(lineafinal)
        ciudad_actual = ciudad
        max_temp = temp


Overwriting reducer1.py


In [4]:
!hdfs dfs -mkdir /pro402
!hdfs dfs -put city_temperature.csv /pro402

mkdir: `/pro402': File exists
put: `/pro402/city_temperature.csv': File exists


In [5]:
!hdfs dfs -rm -r /salida/pro402

Deleted /salida/pro402


In [6]:
!hadoop jar  /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.4.0.jar -file mapper1.py -file reducer1.py -mapper mapper1.py -reducer reducer1.py -input /pro402/city_temperature.csv -output /salida/pro402

2025-12-01 16:42:21,257 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [mapper1.py, reducer1.py, /tmp/hadoop-unjar8407809112403816466/] [] /tmp/streamjob7231373506621982364.jar tmpDir=null
2025-12-01 16:42:21,834 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.7:8032
2025-12-01 16:42:21,943 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.7:8032
2025-12-01 16:42:22,177 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1764606081400_0011
2025-12-01 16:42:22,476 INFO mapred.FileInputFormat: Total input files to process : 1
2025-12-01 16:42:22,489 INFO net.NetworkTopology: Adding a new node: /default-rack/172.20.0.5:9866
2025-12-01 16:42:22,489 INFO net.NetworkTopology: Adding a new node: /default-rack/172.20.0.6:9866
2025-12-01 16:42:22,489 INFO net.Network

In [7]:
!hdfs dfs -cat /salida/pro402/part-00000

Abidjan  --->  88.6	
Abilene  --->  94.2	
Abu Dhabi  --->  107.3	
Addis Ababa  --->  77.0	
Akron Canton  --->  86.1	
Albany  --->  88.0	
Albuquerque  --->  89.4	
Algiers  --->  96.6	
Allentown  --->  91.1	
Almaty  --->  90.9	
Amarillo  --->  92.5	
Amman  --->  95.4	
Amsterdam  --->  85.5	
Anchorage  --->  75.3	
Ankara  --->  87.9	
Antananarivo  --->  95.2	
Ashabad  --->  102.2	
Asheville  --->  85.1	
Athens  --->  94.3	
Atlanta  --->  92.8	
Atlantic City  --->  93.3	
Auckland  --->  75.4	
Austin  --->  94.5	
Baltimore  --->  91.9	
Bangkok  --->  93.0	
Bangui  --->  93.7	
Banjul  --->  93.6	
Barcelona  --->  86.6	
Baton Rouge  --->  90.2	
Beijing  --->  92.9	
Beirut  --->  91.1	
Belfast  --->  77.6	
Belgrade  --->  91.9	
Belize City  --->  92.9	
Bern  --->  83.7	
Bilbao  --->  94.6	
Billings  --->  90.6	
Birmingham  --->  91.0	
Bishkek  --->  91.1	
Bismarck  --->  91.7	
Bissau  --->  100.1	
Bogota  --->  66.7	
Boise  --->  94.2	
Bombay (Mumbai)  --->  92.6	
Bonn  --->  86.9	
Bordeaux  -

# Ejercicio 2: media histórica por país


In [8]:
%%writefile mapper2.py
#!/usr/bin/env python3


import sys


first = True

for line in  sys.stdin:
    
    if first == True:
        first = False
        continue

    line = line.strip().split(',')
    print(f"{line[1]}\t{line[7]}")    



Overwriting mapper2.py


In [9]:
%%writefile reducer2.py
#!/usr/bin/env python3
 
import sys

temp_acumulada = 0
media = 0
pais = None
pais_actual = None
lineafinal = ""
cont = 0

for line in sys.stdin:
    line = line.split('\t',1)

    pais = line[0]
    temp = float(line[1])

    if pais_actual is None:
        pais_actual = pais
        
    temp_acumulada += temp
    cont += 1

    if pais_actual != pais:
        media = temp_acumulada / cont
        lineafinal = f"{pais_actual}  --->  {round(media,2)}"
        cont = 0
        temp_acumulada = 0
        media = 0 
        pais_actual = pais
        print(lineafinal)


Overwriting reducer2.py


In [10]:
!hdfs dfs -rm -r /salida/pro402

Deleted /salida/pro402


In [11]:
!hadoop jar  /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.4.0.jar -file mapper2.py -file reducer2.py -mapper mapper2.py -reducer reducer2.py -input /pro402/city_temperature.csv -output /salida/pro402

2025-12-01 16:42:46,061 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [mapper2.py, reducer2.py, /tmp/hadoop-unjar4821044921523181762/] [] /tmp/streamjob4420737980676491011.jar tmpDir=null
2025-12-01 16:42:46,649 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.7:8032
2025-12-01 16:42:46,785 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.7:8032
2025-12-01 16:42:46,990 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1764606081400_0012
2025-12-01 16:42:47,344 INFO mapred.FileInputFormat: Total input files to process : 1
2025-12-01 16:42:47,357 INFO net.NetworkTopology: Adding a new node: /default-rack/172.20.0.6:9866
2025-12-01 16:42:47,357 INFO net.NetworkTopology: Adding a new node: /default-rack/172.20.0.4:9866
2025-12-01 16:42:47,357 INFO net.Network

In [12]:
!hdfs dfs -cat /salida/pro402/part-00000

Albania  --->  33.18	
Algeria  --->  63.75	
Argentina  --->  62.31	
Australia  --->  61.63	
Austria  --->  51.05	
Bahamas  --->  76.57	
Bahrain  --->  80.63	
Bangladesh  --->  10.11	
Barbados  --->  77.0	
Belarus  --->  41.82	
Belgium  --->  51.06	
Belize  --->  73.48	
Benin  --->  76.15	
Bermuda  --->  66.97	
Bolivia  --->  44.87	
Brazil  --->  70.13	
Bulgaria  --->  45.19	
Burundi  --->  -65.37	
Canada  --->  42.01	
Central African Republic  --->  67.02	
China  --->  59.98	
Colombia  --->  55.25	
Congo  --->  69.32	
Costa Rica  --->  70.37	
Croatia  --->  46.93	
Cuba  --->  72.62	
Cyprus  --->  23.8	
Czech Republic  --->  47.6	
Denmark  --->  46.96	
Dominican Republic  --->  65.2	
Egypt  --->  71.97	
Equador  --->  59.94	
Ethiopia  --->  25.45	
Finland  --->  42.25	
France  --->  54.7	
Gabon  --->  70.4	
Gambia  --->  59.66	
Georgia  --->  46.49	
Germany  --->  6.43	
Greece  --->  59.64	
Guatemala  --->  56.91	
Guinea  --->  49.39	
Guinea-Bissau  --->  2.37	
Guyana  --->  -22.07	
Hai

In [13]:
%%writefile mapper3.py
#!/usr/bin/env python3

import sys

first = True

for line in sys.stdin:
    if first == True:
        first = False
        continue
    line = line.strip().split(',')
    print(f"{line[3]},{line[6]},{line[7]}")    

Overwriting mapper3.py


In [14]:
%%writefile reducer3.py
#!/usr/bin/env python3
 
import sys

ciudad = None
ciudad_actual = None
año_actual = None
lineafinal = ""
cont = 0

for line in sys.stdin:
    line = line.split(',',2)

    ciudad = line[0]
    temp = float(line[2])
    año = float(line[1])

    if ciudad_actual is None:
        ciudad_actual = ciudad

    if año_actual is None:
        año_actual = año
        
    if temp >= 30:
        cont += 1    

    
    if ciudad_actual != ciudad:
        cont = 0
        ciudad_actual = ciudad
    
    if año_actual != año:
        print(f"{año_actual}")
        lineafinal = f"Dias calurosos en  {ciudad_actual}  --->  {cont}"
        print(lineafinal)
        cont = 0
        año_actual = año


Overwriting reducer3.py


In [15]:
!hdfs dfs -rm -r /salida/pro402

Deleted /salida/pro402


In [16]:
!hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.4.0.jar -file mapper3.py -file reducer3.py -mapper mapper3.py -reducer reducer3.py -input /pro402/city_temperature.csv -output /salida/pro402

2025-12-01 16:43:09,136 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [mapper3.py, reducer3.py, /tmp/hadoop-unjar3719127132195960578/] [] /tmp/streamjob5811067307427702411.jar tmpDir=null
2025-12-01 16:43:09,726 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.7:8032
2025-12-01 16:43:09,849 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.7:8032
2025-12-01 16:43:10,018 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1764606081400_0013
2025-12-01 16:43:10,345 INFO mapred.FileInputFormat: Total input files to process : 1
2025-12-01 16:43:10,358 INFO net.NetworkTopology: Adding a new node: /default-rack/172.20.0.5:9866
2025-12-01 16:43:10,359 INFO net.NetworkTopology: Adding a new node: /default-rack/172.20.0.6:9866
2025-12-01 16:43:10,359 INFO net.Network

In [17]:
!hdfs dfs -cat /salida/pro402/part-00000

1995.0	
Dias calurosos en  Abidjan  --->  364	
1996.0	
Dias calurosos en  Abidjan  --->  363	
1997.0	
Dias calurosos en  Abidjan  --->  358	
1998.0	
Dias calurosos en  Abidjan  --->  361	
1999.0	
Dias calurosos en  Abidjan  --->  326	
2000.0	
Dias calurosos en  Abidjan  --->  349	
2001.0	
Dias calurosos en  Abidjan  --->  360	
2002.0	
Dias calurosos en  Abidjan  --->  293	
2003.0	
Dias calurosos en  Abidjan  --->  301	
2004.0	
Dias calurosos en  Abidjan  --->  350	
2005.0	
Dias calurosos en  Abidjan  --->  364	
2006.0	
Dias calurosos en  Abidjan  --->  364	
2007.0	
Dias calurosos en  Abidjan  --->  364	
2008.0	
Dias calurosos en  Abidjan  --->  363	
2009.0	
Dias calurosos en  Abidjan  --->  365	
2010.0	
Dias calurosos en  Abidjan  --->  364	
2011.0	
Dias calurosos en  Abidjan  --->  352	
2012.0	
Dias calurosos en  Abidjan  --->  366	
2013.0	
Dias calurosos en  Abidjan  --->  364	
2014.0	
Dias calurosos en  Abidjan  --->  364	
2015.0	
Dias calurosos en  Abidjan  --->  364	
2016.0	
Dias 

In [18]:
%%writefile mapper4.py
#!/usr/bin/env python3 

import sys

first = True

for line in sys.stdin:
    if first == True:
        first = False
        continue
    
    line = line.strip().split(',')
    print(f"{line[0]}\t{line[7]}")




Overwriting mapper4.py


In [19]:
%%writefile reducer4.py
#!/usr/bin/env python3

import sys

region_actual = None
max_temp = -10000000000
min_temp = 10000000000

for line in sys.stdin:
    line = line.split('\t',1)
    region = line[0]
    temp = float(line[1])

    if region_actual is None:
        region_actual = region
    
    if temp >= max_temp:
        max_temp = temp
    
    if temp < min_temp:
        min_temp = temp

    if region_actual != region:
        print(f"{region_actual} temperatura máxima {max_temp}, temperatura mínima {min_temp}")
        region_actual = region
        max_temp = -10000000000
        min_temp = 10000000000


Overwriting reducer4.py


In [20]:
!hdfs dfs -rm -r /salida/pro402

Deleted /salida/pro402


In [21]:
!hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.4.0.jar -file mapper4.py -file reducer4.py -mapper mapper4.py -reducer reducer4.py -input /pro402/city_temperature.csv -output /salida/pro402

2025-12-01 16:43:33,177 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [mapper4.py, reducer4.py, /tmp/hadoop-unjar6286132611876431684/] [] /tmp/streamjob5902414792359543308.jar tmpDir=null
2025-12-01 16:43:33,797 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.7:8032
2025-12-01 16:43:33,921 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.7:8032
2025-12-01 16:43:34,099 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1764606081400_0014
2025-12-01 16:43:34,468 INFO mapred.FileInputFormat: Total input files to process : 1
2025-12-01 16:43:34,483 INFO net.NetworkTopology: Adding a new node: /default-rack/172.20.0.5:9866
2025-12-01 16:43:34,483 INFO net.NetworkTopology: Adding a new node: /default-rack/172.20.0.4:9866
2025-12-01 16:43:34,484 INFO net.Network

In [22]:
!hdfs dfs -cat /salida/pro402/part-00000

Africa temperatura máxima 102.8, temperatura mínima -99.0	
Asia temperatura máxima 103.7, temperatura mínima -99.0	
Australia/South Pacific temperatura máxima 96.8, temperatura mínima -99.0	
Europe temperatura máxima 102.5, temperatura mínima -99.0	
Middle East temperatura máxima 110.0, temperatura mínima -99.0	
North America temperatura máxima 107.7, temperatura mínima -99.0	
